In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 128
tree_depth = 8
batch_size = 512
device = 'cpu'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [ ]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.479093551635742 | KNN Loss: 5.855673789978027 | CLS Loss: 1.623420000076294
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 6.210329532623291 | KNN Loss: 5.366710186004639 | CLS Loss: 0.843619167804718
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 5.910048961639404 | KNN Loss: 5.182383060455322 | CLS Loss: 0.7276658415794373
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 5.910216808319092 | KNN Loss: 5.185675621032715 | CLS Loss: 0.7245411276817322
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 5.7453765869140625 | KNN Loss: 5.1567535400390625 | CLS Loss: 0.5886231064796448
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 5.7135725021362305 | KNN Loss: 5.165256500244141 | CLS Loss: 0.5483161807060242
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 5.619647979736328 | KNN Loss: 5.102312088012695 | CLS Loss: 0.5173361301422119
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 5.560379505157471 | KNN Loss: 5.040430068969727 | CLS L

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 4.985884189605713 | KNN Loss: 4.904341697692871 | CLS Loss: 0.08154263347387314
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 4.954887390136719 | KNN Loss: 4.829421043395996 | CLS Loss: 0.1254662275314331
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 5.042200565338135 | KNN Loss: 4.883302211761475 | CLS Loss: 0.15889844298362732
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 4.958935737609863 | KNN Loss: 4.862678050994873 | CLS Loss: 0.09625766426324844
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 4.986711025238037 | KNN Loss: 4.874623775482178 | CLS Loss: 0.11208714544773102
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 5.0000319480896 | KNN Loss: 4.879295825958252 | CLS Loss: 0.12073612213134766
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 4.962998390197754 | KNN Loss: 4.871077537536621 | CLS Loss: 0.09192091226577759
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 4.993232727050781 | KNN Loss: 4.8496975898742

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 4.893457889556885 | KNN Loss: 4.822717666625977 | CLS Loss: 0.07074011117219925
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 4.861776828765869 | KNN Loss: 4.796504974365234 | CLS Loss: 0.06527180969715118
Epoch: 007, Loss: 4.8884, Train: 0.9793, Valid: 0.9767, Best: 0.9767
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 4.8307671546936035 | KNN Loss: 4.7627339363098145 | CLS Loss: 0.06803344935178757
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 4.84659481048584 | KNN Loss: 4.772560119628906 | CLS Loss: 0.07403487712144852
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 4.882425308227539 | KNN Loss: 4.782869338989258 | CLS Loss: 0.09955620020627975
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 4.863083839416504 | KNN Loss: 4.797335147857666 | CLS Loss: 0.06574861705303192
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 4.869645118713379 | KNN Loss: 4.79896879196167 | CLS Loss: 0.07067614048719406
Epoch 8 / 200 | iteration 5

Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 4.899861812591553 | KNN Loss: 4.8376240730285645 | CLS Loss: 0.06223784387111664
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 4.78014612197876 | KNN Loss: 4.730955600738525 | CLS Loss: 0.04919062554836273
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 4.873658180236816 | KNN Loss: 4.794013977050781 | CLS Loss: 0.07964427769184113
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 4.814882755279541 | KNN Loss: 4.77776575088501 | CLS Loss: 0.03711717575788498
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 4.84725284576416 | KNN Loss: 4.7870941162109375 | CLS Loss: 0.06015869230031967
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 4.857163906097412 | KNN Loss: 4.77741813659668 | CLS Loss: 0.07974568754434586
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 4.795465469360352 | KNN Loss: 4.742465019226074 | CLS Loss: 0.053000494837760925
Epoch 11 / 200 | iteration 130 / 171 | Total Loss: 4.8629584312438965 | KNN Loss: 4.7930

Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 4.8884992599487305 | KNN Loss: 4.823034763336182 | CLS Loss: 0.06546467542648315
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 4.751100540161133 | KNN Loss: 4.7270307540893555 | CLS Loss: 0.02406957373023033
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 4.875583171844482 | KNN Loss: 4.772170543670654 | CLS Loss: 0.10341261327266693
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 4.816015720367432 | KNN Loss: 4.763940334320068 | CLS Loss: 0.052075307816267014
Epoch: 014, Loss: 4.8055, Train: 0.9863, Valid: 0.9821, Best: 0.9823
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 4.829922199249268 | KNN Loss: 4.766785144805908 | CLS Loss: 0.0631370022892952
Epoch 15 / 200 | iteration 10 / 171 | Total Loss: 4.83040189743042 | KNN Loss: 4.762335777282715 | CLS Loss: 0.06806630641222
Epoch 15 / 200 | iteration 20 / 171 | Total Loss: 4.758620738983154 | KNN Loss: 4.724384307861328 | CLS Loss: 0.03423648327589035
Epoch 15 / 200 | ite

...

Epoch 89 / 200 | iteration 100 / 171 | Total Loss: 4.696671962738037 | KNN Loss: 4.683212757110596 | CLS Loss: 0.013459126465022564
Epoch 89 / 200 | iteration 110 / 171 | Total Loss: 4.685176849365234 | KNN Loss: 4.669690132141113 | CLS Loss: 0.01548655703663826
Epoch 89 / 200 | iteration 120 / 171 | Total Loss: 4.692789077758789 | KNN Loss: 4.681221961975098 | CLS Loss: 0.011567292734980583
Epoch 89 / 200 | iteration 130 / 171 | Total Loss: 4.678859233856201 | KNN Loss: 4.669167518615723 | CLS Loss: 0.009691828861832619
Epoch 89 / 200 | iteration 140 / 171 | Total Loss: 4.731479167938232 | KNN Loss: 4.70622444152832 | CLS Loss: 0.025254879146814346
Epoch 89 / 200 | iteration 150 / 171 | Total Loss: 4.6768083572387695 | KNN Loss: 4.667814254760742 | CLS Loss: 0.008994298987090588
Epoch 89 / 200 | iteration 160 / 171 | Total Loss: 4.726377964019775 | KNN Loss: 4.714968681335449 | CLS Loss: 0.011409463360905647
Epoch 89 / 200 | iteration 170 / 171 | Total Loss: 4.695120334625244 | KNN Lo

Epoch 92 / 200 | iteration 170 / 171 | Total Loss: 4.696181297302246 | KNN Loss: 4.677591800689697 | CLS Loss: 0.01858963444828987
Epoch: 092, Loss: 4.7105, Train: 0.9964, Valid: 0.9859, Best: 0.9872
Epoch 93 / 200 | iteration 0 / 171 | Total Loss: 4.6847028732299805 | KNN Loss: 4.680771827697754 | CLS Loss: 0.003931147512048483
Epoch 93 / 200 | iteration 10 / 171 | Total Loss: 4.655590534210205 | KNN Loss: 4.651773452758789 | CLS Loss: 0.0038171089254319668
Epoch 93 / 200 | iteration 20 / 171 | Total Loss: 4.789667129516602 | KNN Loss: 4.769301414489746 | CLS Loss: 0.020365657284855843
Epoch 93 / 200 | iteration 30 / 171 | Total Loss: 4.728115081787109 | KNN Loss: 4.700418472290039 | CLS Loss: 0.02769654430449009
Epoch 93 / 200 | iteration 40 / 171 | Total Loss: 4.697596073150635 | KNN Loss: 4.6905694007873535 | CLS Loss: 0.007026592269539833
Epoch 93 / 200 | iteration 50 / 171 | Total Loss: 4.691486835479736 | KNN Loss: 4.673923015594482 | CLS Loss: 0.017563797533512115
Epoch 93 / 20

Epoch 96 / 200 | iteration 60 / 171 | Total Loss: 4.715567588806152 | KNN Loss: 4.693706035614014 | CLS Loss: 0.021861374378204346
Epoch 96 / 200 | iteration 70 / 171 | Total Loss: 4.68407678604126 | KNN Loss: 4.6768059730529785 | CLS Loss: 0.007270941976457834
Epoch 96 / 200 | iteration 80 / 171 | Total Loss: 4.700981616973877 | KNN Loss: 4.687576770782471 | CLS Loss: 0.01340483222156763
Epoch 96 / 200 | iteration 90 / 171 | Total Loss: 4.694089889526367 | KNN Loss: 4.668557167053223 | CLS Loss: 0.02553291618824005
Epoch 96 / 200 | iteration 100 / 171 | Total Loss: 4.676794052124023 | KNN Loss: 4.664745807647705 | CLS Loss: 0.01204825658351183
Epoch 96 / 200 | iteration 110 / 171 | Total Loss: 4.658322811126709 | KNN Loss: 4.649949550628662 | CLS Loss: 0.008373203687369823
Epoch 96 / 200 | iteration 120 / 171 | Total Loss: 4.710494518280029 | KNN Loss: 4.7052693367004395 | CLS Loss: 0.005225078668445349
Epoch 96 / 200 | iteration 130 / 171 | Total Loss: 4.706723213195801 | KNN Loss: 4

Epoch 99 / 200 | iteration 130 / 171 | Total Loss: 4.736800670623779 | KNN Loss: 4.686923980712891 | CLS Loss: 0.04987671226263046
Epoch 99 / 200 | iteration 140 / 171 | Total Loss: 4.702376365661621 | KNN Loss: 4.692188739776611 | CLS Loss: 0.010187593288719654
Epoch 99 / 200 | iteration 150 / 171 | Total Loss: 4.709314346313477 | KNN Loss: 4.698884963989258 | CLS Loss: 0.01042957417666912
Epoch 99 / 200 | iteration 160 / 171 | Total Loss: 4.7336812019348145 | KNN Loss: 4.706918716430664 | CLS Loss: 0.02676249109208584
Epoch 99 / 200 | iteration 170 / 171 | Total Loss: 4.706286907196045 | KNN Loss: 4.699099063873291 | CLS Loss: 0.007187775336205959
Epoch: 099, Loss: 4.7095, Train: 0.9960, Valid: 0.9858, Best: 0.9872
Epoch 100 / 200 | iteration 0 / 171 | Total Loss: 4.689737796783447 | KNN Loss: 4.677109241485596 | CLS Loss: 0.012628618627786636
Epoch 100 / 200 | iteration 10 / 171 | Total Loss: 4.661083698272705 | KNN Loss: 4.643423557281494 | CLS Loss: 0.01766018196940422
Epoch 100 /

Epoch 103 / 200 | iteration 10 / 171 | Total Loss: 4.678947925567627 | KNN Loss: 4.6758270263671875 | CLS Loss: 0.0031210239976644516
Epoch 103 / 200 | iteration 20 / 171 | Total Loss: 4.65442419052124 | KNN Loss: 4.6470770835876465 | CLS Loss: 0.007346881553530693
Epoch 103 / 200 | iteration 30 / 171 | Total Loss: 4.6808648109436035 | KNN Loss: 4.672391891479492 | CLS Loss: 0.008472945541143417
Epoch 103 / 200 | iteration 40 / 171 | Total Loss: 4.759029388427734 | KNN Loss: 4.7434797286987305 | CLS Loss: 0.015549637377262115
Epoch 103 / 200 | iteration 50 / 171 | Total Loss: 4.728278160095215 | KNN Loss: 4.719176292419434 | CLS Loss: 0.009101921692490578
Epoch 103 / 200 | iteration 60 / 171 | Total Loss: 4.678065776824951 | KNN Loss: 4.6706109046936035 | CLS Loss: 0.007454941049218178
Epoch 103 / 200 | iteration 70 / 171 | Total Loss: 4.651780605316162 | KNN Loss: 4.64718770980835 | CLS Loss: 0.0045930808410048485
Epoch 103 / 200 | iteration 80 / 171 | Total Loss: 4.657245635986328 | 

In [ ]:
test(model, test_data_iter, device)

In [ ]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

In [ ]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

In [ ]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [ ]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

In [ ]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [ ]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [ ]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [ ]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [ ]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [ ]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [ ]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")